<a href="https://colab.research.google.com/github/manoj-b-22/IDS-in-IoV/blob/main/CNN_inter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Conv1D, Dense, AveragePooling1D
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
scaler = StandardScaler()
encoder = LabelEncoder()

In [4]:
train = pd.read_csv('/content/drive/MyDrive/train-intervehicle.csv',chunksize=500000)
test = pd.read_csv('/content/drive/MyDrive/test-intervehicle.csv')

In [5]:
model = Sequential()
model.add(Conv1D(activation="sigmoid", input_shape=(len(test.columns)-1,1), kernel_size=2, filters=100))
model.add(AveragePooling1D(pool_size=2))
model.add(Dense(1))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 66, 6)             36        
                                                                 
 average_pooling1d (AverageP  (None, 33, 6)            0         
 ooling1D)                                                       
                                                                 
 conv1d_1 (Conv1D)           (None, 29, 6)             186       
                                                                 
 average_pooling1d_1 (Averag  (None, 14, 6)            0         
 ePooling1D)                                                     
                                                                 
 conv1d_2 (Conv1D)           (None, 10, 12)            372       
                                                                 
 average_pooling1d_2 (Averag  (None, 5, 12)            0

In [6]:
for chunk in train:
  train_y = chunk.pop('Label')
  train_x = chunk
  train_x = scaler.fit_transform(train_x)
  train_y = encoder.fit_transform(train_y)
  model.fit(train_x,train_y,batch_size=32,epochs=6)

6161/6161 [==============================] - 20s 3ms/step - loss: 7.5155 - accuracy: 0.5072


In [7]:
test_y = test.pop('Label')
test_x = test

In [8]:
test_x = scaler.transform(test_x)
test_y = encoder.transform(test_y)

In [9]:
score,accuracy = model.evaluate(test_x,test_y)
print('Score',score,"Accuracy",accuracy)

79666/79666 [==============================] - 234s 3ms/step - loss: 7.5089 - accuracy: 0.5074
Score 7.508892059326172 Accuracy 0.5074149966239929
